<a href="https://colab.research.google.com/github/rongxie2023/dlcv/blob/main/Chapter02//home/xr/dlcv/Chapter02/Chapter02/11-case5-linear_regression_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 设置随机种子以获得可重复的结果
torch.manual_seed(0)

# 准备数据集
n_samples = 100
w = torch.tensor([2, -3.4])
b = 4.2
# X = torch.rand((n_samples, len(w)))  #均匀分布         ***设为均匀分布后，注意设置训练轮次
X = torch.normal(0, 1, (n_samples, len(w))) #正态分布
y = X @ w +b +torch.normal(0, 0.01, [n_samples])
y = y.reshape(-1,1)

print(X.shape,y.shape)

In [ ]:
plt.rcParams['figure.figsize'] = [3, 3] 
xx=X[:, (1)].detach().numpy()
yy=y.detach().numpy()
plt.scatter(xx, yy, 1)

In [ ]:
#直接根据定义求w
# 创建一个全为 1 的列，形状为 (3, 1)
ones_column = torch.ones(X.shape[0], 1)
XX=torch.cat((X, ones_column), dim=1).numpy()
yy=y.numpy()
ww=np.linalg.inv(XX.T@XX)@XX.T@yy
print(ww)

#纯torch实现
ones_column = torch.ones(X.shape[0], 1)
XX=torch.cat((X, ones_column), dim=1)
ww=torch.inverse(XX.T@XX)@XX.T@y
print(ww)


In [ ]:

from torch.utils import data
dataset = data.TensorDataset(*(X,y))  ###相当于上面SImpleDataset
# 使用DataLoader加载数据
data_loader = DataLoader(dataset, batch_size=10, shuffle=True)

# 定义模型
model = nn.Sequential(
    nn.Linear(2, 1)  # 输入特征数为2，输出为1
)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# 训练模型
# num_epochs = 2000
# update_interval = 100
num_epochs = 300
update_interval = 10

# 使用TensorBoard记录训练过程
writer = SummaryWriter()

for epoch in tqdm(range(num_epochs)):
    # 初始化epoch损失为0
    epoch_loss = 0.0
        
    # 每个epoch遍历整个数据集一次
    for i, (inputs, labels) in enumerate(data_loader):
        # 前向传播
        outputs = model(inputs)

        # 计算损失
        loss = criterion(outputs, labels)

        # 更新epoch损失
        epoch_loss += loss.item()

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 使用TensorBoard记录损失
        writer.add_scalar('Loss', loss.item(), epoch)

    if (epoch+1) %  update_interval == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {epoch_loss / len(data_loader):.4f}')

# 打印最终的权重和偏置
print('Final weights and bias: ', list(model.parameters()))

在命令行执行以下命令： 
```
tensorboard --logdir runs
```
然后在Web浏览器中打开显示的URL地址，通常为 http://localhost:6006/ ，在那里你可以看到训练过程中损失的变化。

### 实验结果

In [ ]:
print('real weights',w,b)
print('Final weights and bias: ', list(model.parameters()))
model.eval()  #查看文档，思考eval()的作用

test_x=torch.zeros((10, len(w))).uniform_(-2,2)
with torch.no_grad():
    test_y=model(test_x)   ###获取网络的输出

real_y=test_x @ w +b  ###计算真实值
print(test_y)
print(real_y)


